In [ ]:
-- CHANGE THIS WHEN MOVING TO IFF ENVIRONMENT
USE DATABASE SCENT_WRK_DEV_DB;
USE SCHEMA DEMO;

In [ ]:




CREATE OR REPLACE VIEW FACE_CARE_PRODUCT_AGG_BY_SKIN_TYPE
AS
(
SELECT 
      selling_name as "PRODUCT"
    , skin_type_face as "SKIN-TYPE"
    , avg(liking) AS "AVG-LIKING"  
    , avg(CM_SUCCESSFUl) AS "AVG-SUCCESS"
    , avg(FREQ_FACIAL_MASK) AS "FREQ-FAC-MASK"
    , avg(FREQ_FACIAL_TONER) AS "FREQ-FAC-TON"
FROM SCENT_WRK_DEV_DB.DEMO.pilot_1_product_test
WHERE END_USE ILIKE 'Face Care Products'
GROUP BY selling_name, skin_type_face
)

    
    

In [ ]:
SELECT *
FROM FACE_CARE_PRODUCT_AGG_BY_SKIN_TYPE;

In [ ]:
SELECT TOP 100 * FROM SCENT_WRK_DEV_DB.DEMO.pilot_1_e_commerce_comments;

CREATE OR REPLACE VIEW Topic_Mentions_by_Product
as (
    with parsed as 
    (
    select parse_json(topics) as p_topics, *
    from SCENT_WRK_DEV_DB.DEMO.pilot_1_e_commerce_comments
    ),
    flattened as
    (select 
          p.p_topics:positive as positive
        , p.p_topics:"mentionned_topics" as mentioned_list
        , topic.value as topic
        , p.*
    from parsed p
    , LATERAL FLATTEN (input=>p_topics:mentionned_topics) m
    , LATERAL FLATTEN(strtok_to_array(m.value, ', ')) topic
    )
    select 
        name_voysen,
        topic,
        count(*) as "Number of Mentions"
    from flattened
    GROUP BY name_voysen, topic
);

In [ ]:
-- Welcome to Snowflake Notebooks!
select 
     selling_name AS "PRODUCT NAME"
     , skin_type_face AS "SKIN TYPE"
     , avg(liking) AS "AVERAGE LIKING"
 from pilot_1_product_test
 WHERE SKIN_TYPE_FACE IS NOT NULL
 and END_USE ILIKE 'Face Care Products'
 GROUP BY selling_name, skin_type_face --, cluster
 ORDER BY selling_name, skin_type_face --, cluster
 ;

In [ ]:
select 
    
    Name_Voysen as Product, 
    Topic, 
    "Number of Mentions" as Mentions
    
from Topic_Mentions_by_Product
QUALIFY RANK() OVER (PARTITION BY Name_Voysen ORDER BY "Number of Mentions" DESC) <=3
order by Name_Voysen, "Number of Mentions" DESC;


In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


In [ ]:
# Then, we can use the python name to turn cell2 into a Pandas dataframe
my_df = AVG_LIKING_BY_SKIN_TYPE.to_pandas()

# Chart the data
st.subheader("PRODUCT Average liking by Skin Type Face")
st.bar_chart(my_df, x='PRODUCT NAME', y='AVERAGE LIKING', color='SKIN TYPE', stack=False)



In [ ]:

my_df = TOPICS_MENTIONED_BY_PRODUCT.to_pandas()

# Chart the data
st.subheader("Top 3 Topics Mentioned by Product")
st.bar_chart(my_df, x="PRODUCT", y="MENTIONS", color='TOPIC', stack=False)

